In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"
import gym
import argparse
from ddpg_torch.ddpg_torch import Agent
import gym
import numpy as np
from lifting_rl.linkage_env import LinkageEnv

from livelossplot import PlotLosses

In [2]:
params = {
    "N_LINKS": 5,
    "INIT_STATE": np.array([np.pi / 2, np.pi/2, np.pi/2, -np.pi/4, np.pi/4, 0, 0, 0, 0, 0], dtype=np.float32),
    "PARAM_VALS": np.array([9.81, 0.4, 1, 0.4, 1, 0.6, 1, 0.4, 1, 0.4, 1]),
    "OBS_LOW": np.array([0, np.pi/4, -np.pi/2, -np.pi, -np.pi, -8, -8, -8, -8, -8], dtype=np.float32),
    "OBS_HIGH": np.array(
        [3*np.pi/4, 3*np.pi/2, 3*np.pi/4, np.pi/2, np.pi, 8, 8, 8, 8, 8], dtype=np.float32
    ),
    "ACT_LOW": -100,
    "ACT_HIGH": 100,
    "TIME_STEP": 0.01,
    "VIDEO_FPS": 30,
}

In [3]:
np.random.seed(123)

In [4]:
angles_file = "/home/mans/git/human-body-model-dynamics/data/skeleton_angles.csv"
env = LinkageEnv(angles_file, params, verbose=0)

\operatorname{u_{0}}{\left(t \right)}\mathbf{\hat{n}_z}
\frac{l_{0} \operatorname{u_{0}}{\left(t \right)}}{2}\mathbf{\hat{rf0}_y}
------------------
(\operatorname{u_{0}}{\left(t \right)} + \operatorname{u_{1}}{\left(t \right)})\mathbf{\hat{n}_z}
l_{0} \operatorname{u_{0}}{\left(t \right)}\mathbf{\hat{rf0}_y} + \frac{l_{1} \left(\operatorname{u_{0}}{\left(t \right)} + \operatorname{u_{1}}{\left(t \right)}\right)}{2}\mathbf{\hat{rf1}_y}
------------------
(\operatorname{u_{0}}{\left(t \right)} + \operatorname{u_{1}}{\left(t \right)} + \operatorname{u_{2}}{\left(t \right)})\mathbf{\hat{n}_z}
l_{1} \left(\operatorname{u_{0}}{\left(t \right)} + \operatorname{u_{1}}{\left(t \right)}\right)\mathbf{\hat{rf1}_y} + l_{0} \operatorname{u_{0}}{\left(t \right)}\mathbf{\hat{rf0}_y} + \frac{l_{2} \left(\operatorname{u_{0}}{\left(t \right)} + \operatorname{u_{1}}{\left(t \right)} + \operatorname{u_{2}}{\left(t \right)}\right)}{2}\mathbf{\hat{rf2}_y}
------------------
(\operatorname{u_{0}}{\left(t \r

KeyboardInterrupt: 

In [ ]:
agent = Agent(
    lr_actor=0.000025,
    lr_critic=0.00025,
    input_dims=[15],
    tau=0.001,
    env=env,
    batch_size=64,
    layer1_size=400,
    layer2_size=300,
    n_actions=1,
)

In [ ]:
liveloss = PlotLosses()
score_history = []

In [ ]:
for i in range(100000):
    done = False
    score = 0
    obs = env.reset()
    agent.noise.reset()
    while not done:
        env.render()
        act = agent.choose_action(obs)
        new_state, reward, done, info = env.step(act)
        agent.remember(obs, act, reward, new_state, int(done))
        agent.learn()
        score += reward
        obs = new_state

    score_history.append(score)

    metrics = {"score_history": np.mean(score_history[-100:])}
    liveloss.update(metrics)
    liveloss.send()
    print(
        "episode",
        i,
        "score %.2f" % score,
        "100 game average %.2f" % np.mean(score_history[-100:]),
    )
env.close()